In [1]:
from StormExecutor import StormExecutor, _build_sensor_selection_const, _build_world_definition_const, \
    _define_program_constants, _configure_buildfull_options
from builders.POMDPAdapter import POMDPAdapter
from builders.ssp import LineTPMC
import stormpy
import stormpy.pomdp
import stormpy.pars

tpmc = LineTPMC(budget=30, goal=30, length=61)
pomdp = POMDPAdapter(tpmc)
exec = StormExecutor(verbose=True, puzzle_type=pomdp.puzzle_type)


obs_function = [1] + [1]*29 + [-1] + [0]*30
constants = {
    **_build_sensor_selection_const(obs_function, 50),
    **_build_world_definition_const(pomdp),
}

# Instantiate all constant parameters in the PRISM model
static_program = _define_program_constants(exec.static_program, constants)

# Build the parsed POMDP model with full labels and rewards (make it canonic)
build_options = _configure_buildfull_options()
model = stormpy.build_sparse_exact_model_with_options(static_program, build_options)
model = stormpy.build_sparse_model_with_options(static_program, build_options)
model = stormpy.pomdp.make_canonic(model)

In [2]:
# Convert POMDP to parametric model using unknown FSC
parametric_model = stormpy.pomdp.apply_unknown_fsc(
    model,
    stormpy.pomdp.PomdpFscApplicationMode.simple_linear
)

In [ ]:
# Option 1: Parameter synthesis (requires bounded reward property <= tau)
env = stormpy.Environment()
checker = stormpy.pars.create_region_checker(env, parametric_model, exec.property[0].raw_formula)
parameters = parametric_model.collect_probability_parameters()
print(parameters)

valuations = { parameter: (0, 1) for parameter in parameters }
region = stormpy.pars.ParameterRegion(valuations)
synthesis_result = checker.check_region(env, region)

## Using Parametric MC with Gradient Descent for Memory-less POMDP Controllers

To integrate a parametric Markov chain in a Gradient Descent algorithm for computing memory-less POMDP controllers, one needs to combine `stormpy`'s parametric analysis capabilities with optimization logic.

### Core Approach
The key is to use the parametric model from `apply_unknown_fsc` as a differentiable function that maps controller parameters to verification results, then apply gradient descent to optimize these parameters.

Implementation Steps
1. Create Parametric Model with Unknown FSC

In [5]:
# Convert POMDP to parametric model using unknown FSC
parametric_model = stormpy.pomdp.apply_unknown_fsc(
    model,
    stormpy.pomdp.PomdpFscApplicationMode.simple_linear
)

2. Extract Parameters and Derivatives

In [3]:
# Collect model parameters
parameters = parametric_model.collect_probability_parameters()

print(parameters)
print(parametric_model)

# Function to compute gradients
def compute_gradients(parametric_model, parameters):
    gradients = {}
    for param in parameters:
        # Gather derivatives for each parameter
        derivatives = stormpy.pars.gather_derivatives(parametric_model, param)
        gradients[param] = derivatives
    return gradients

print(compute_gradients(parametric_model, parameters))

{<Variable p14_0 [id = 3]>, <Variable p32_0 [id = 4]>, <Variable p22_0 [id = 5]>, <Variable p20_0 [id = 6]>, <Variable p27_0 [id = 7]>, <Variable p28_0 [id = 8]>, <Variable p18_0 [id = 9]>, <Variable p2_0 [id = 10]>, <Variable p29_0 [id = 11]>, <Variable p5_0 [id = 12]>, <Variable p12_0 [id = 13]>, <Variable p8_0 [id = 14]>, <Variable p4_0 [id = 15]>, <Variable p19_0 [id = 16]>, <Variable p21_0 [id = 17]>, <Variable p10_0 [id = 18]>, <Variable p23_0 [id = 19]>, <Variable p6_0 [id = 20]>, <Variable p25_0 [id = 21]>, <Variable p24_0 [id = 22]>, <Variable p11_0 [id = 23]>, <Variable p3_0 [id = 24]>, <Variable p26_0 [id = 25]>, <Variable p7_0 [id = 26]>, <Variable p17_0 [id = 27]>, <Variable p13_0 [id = 28]>, <Variable p30_0 [id = 29]>, <Variable p9_0 [id = 30]>, <Variable p16_0 [id = 31]>, <Variable p31_0 [id = 32]>, <Variable p0_0 [id = 33]>, <Variable p1_0 [id = 34]>, <Variable p1_1 [id = 35]>}
-------------------------------------------------------------- 
Model type: 	DTMC (sparse)
St

In [4]:
import stormpy.pycarl.cln

# Function to evaluate gradients using current values for parameters
def evaluate_gradient(gradient_formulas: set, values: dict):
    if len(gradient_formulas) == 0:
        return 0
    else:
        gradient_polynomial = gradient_formulas.pop()
        grad_val = gradient_polynomial.evaluate(values)
        return float(grad_val)


3. Gradient Descent Optimization Loop

In [8]:
def gradient_descent_optimization(parametric_model, formula, parameters,
                                learning_rate=0.01, max_iterations=100):
    # Initialize parameters
    current_values = {p: stormpy.RationalRF(0.5) for p in parameters}
    instantiator = stormpy.pars.ModelInstantiator(parametric_model)

    for iteration in range(max_iterations):
        # Instantiate model with current parameters
        concrete_model = instantiator.instantiate(current_values)

        # Model check to get objective value
        result = stormpy.model_checking(concrete_model, formula)
        objective = float(result.at(concrete_model.initial_states[0]))

        # Compute gradients (symbolic differentiation)
        gradients = compute_gradients(parametric_model, parameters)

        # Update parameters using gradient descent
        for param in parameters:
            # print(f"Parameter: {param}, Gradient: {gradients[param]}, Value: {float(current_values[param])}")
            # Convert symbolic gradient to numeric
            grad_value = evaluate_gradient(gradients[param], current_values)
            current_values[param] = stormpy.RationalRF(
                float(current_values[param]) - learning_rate * grad_value
            )

        print(f"Iteration {iteration}: Objective = {objective}")

    return current_values


4. Run Gradient Descent - extract memory-less controller

In [21]:
# Check CLN availability
from stormpy import pycarl
if not pycarl.has_cln():
    raise ImportError("CLN backend not available")

In [10]:
# Run gradient descent
optimal_parameters = gradient_descent_optimization(
    parametric_model, exec.property[0].raw_formula, parameters,
    learning_rate=0.05, max_iterations=50
)

# Get final controller
instantiator = stormpy.pars.ModelInstantiator(parametric_model)
optimal_model = instantiator.instantiate(optimal_parameters)
final_result = stormpy.model_checking(optimal_model, exec.property[0].raw_formula)

Iteration 0: Objective = 630.3333333333331
Iteration 1: Objective = 630.3333333333331
Iteration 2: Objective = 630.3333333333331
Iteration 3: Objective = 630.3333333333331
Iteration 4: Objective = 630.3333333333331
Iteration 5: Objective = 630.3333333333331
Iteration 6: Objective = 630.3333333333331
Iteration 7: Objective = 630.3333333333331
Iteration 8: Objective = 630.3333333333331
Iteration 9: Objective = 630.3333333333331
Iteration 10: Objective = 630.3333333333331
Iteration 11: Objective = 630.3333333333331
Iteration 12: Objective = 630.3333333333331
Iteration 13: Objective = 630.3333333333331
Iteration 14: Objective = 630.3333333333331
Iteration 15: Objective = 630.3333333333331
Iteration 16: Objective = 630.3333333333331
Iteration 17: Objective = 630.3333333333331
Iteration 18: Objective = 630.3333333333331
Iteration 19: Objective = 630.3333333333331
Iteration 20: Objective = 630.3333333333331
Iteration 21: Objective = 630.3333333333331
Iteration 22: Objective = 630.333333333333


### Key Components

>   `apply_unknown_fsc`: Creates parametric model with FSC parameters from POMDP

>   ModelInstantiator: Instantiates parametric models with specific values

>   `gather_derivatives`: Computes symbolic derivatives of transition probabilities

>   Parametric model checking: Returns symbolic results for gradient computation

### Memory-less Strategy Enforcement

The `SIMPLE_LINEAR` FSC application mode ensures memory-less strategies by:

- Using a linear parameterization without memory states
- Mapping observations directly to action probabilities
- Enforcing that the controller depends only on current observations

### Notes

- Stormpy provides the parametric model and derivative computation, but the gradient descent algorithm must be implemented in Python
- The `gather_derivatives` function returns symbolic derivatives that need to be evaluated numerically
- Learning rate and convergence criteria should be tuned based on the specific POMDP
- The approach works for reachability and probability properties; reward properties may need different handling
- For complex gradients, consider using automatic differentiation libraries with stormpy's parametric results